In [2]:
import pandas as pd
import numpy as np

import pickle
from pathlib import Path
from tqdm import tqdm

from sentence_transformers import SentenceTransformer

В данном ноутбуке по описаниям книг будут построены векторные предсатвления с помощью трансформеров для задачи Text Similarity. \
Источник модели: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Построенные вектора будут использоваться для поиска похожих авторов / произведений.

In [3]:
full_dataset = pd.concat([pd.read_csv(str(x.resolve())) for x in Path("data/").glob("*k.csv")])

columns = ['Id', 'Name', 'RatingDist1', 'RatingDist2', 'RatingDist3',
           'RatingDist4', 'RatingDist5', 'Rating', 'RatingDistTotal', 'pagesNumber', 'Publisher',
           'Authors', 'Language', 'Description']
full_dataset = full_dataset[columns]

full_dataset = full_dataset[~full_dataset['Description'].isna()]
full_dataset['RatingDistTotal'] = full_dataset['RatingDistTotal'].apply(lambda x: float(x.replace('total:', '')))

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

full_dataset = reduce_mem_usage(full_dataset)

Mem. usage decreased to 113.93 Mb (15.0% reduction)


In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
import re

def text_preprocessing_step(text):
    text = text.lower()

    # Удаляем html-теги
    re_html = re.compile(r'<.*?>')
    text = re_html.sub(' ', text)

    return text.strip()

# Т.к. книг много, ограничим их кол-во, для экономии времени
full_dataset = full_dataset[full_dataset['Description'].apply(len) >= 50]
full_dataset = full_dataset[full_dataset['RatingDistTotal'] >= 2000]

descriptions = full_dataset['Description'].apply(text_preprocessing_step)

In [ ]:
# Строим ембединги, используя загруженную модель.
embeddings = model.encode(descriptions.values)

In [6]:
# Можно загрузить уже посчитанный и сохранённый набор векторов.

with open('result/embeddings_v1.pkl', 'rb') as file:
    embeddings = pickle.load(file)

In [10]:
embeddings.shape, descriptions.shape

((66520, 384), (66520,))

В данном датасете использовались несколько разных языков, оставим только те произведения где язык точно английский, и где он не указан.

In [6]:
filters = (full_dataset['Language'] == 'eng') | (full_dataset['Language'].isna())

embeddings = embeddings[filters]
descriptions = descriptions[filters]
full_dataset = full_dataset[filters]

NameError: name 'embeddings' is not defined

Для каждой книги посчитаем топ 20 ближайших к ней. Сохраним имена этих книг в словарь, который дальше будем использовать в telegram боте.

In [12]:
distances = []
near_indexes = []

for i, emb in tqdm(enumerate(embeddings)):
    distances_tmp = np.dot(embeddings, emb)
    indexes = np.argsort(distances_tmp)[::-1][:20]

    distances.append(distances_tmp[indexes])
    near_indexes.append(indexes)

53221it [13:04, 67.84it/s]


In [61]:
books_dict = {}

for i, idxs in tqdm(enumerate(near_indexes)):
    book_name = full_dataset.iloc[i]['Name']
    similar_books = full_dataset.iloc[idxs]['Name'].drop_duplicates().values
    similar_books = [name for name in similar_books if name != book_name]

    books_dict[book_name] = similar_books

53221it [00:22, 2315.19it/s]


In [62]:
with open('result/book_distances_v1.pkl', 'wb') as file:
    pickle.dump([distances, near_indexes], file)

with open('result/books_dict_v1.pkl', 'wb') as file:
    pickle.dump(books_dict, file)

Далее посчитаем вектор-представление каждого автора, как среднее векторов описаний его книг. \
Для каждого из них найдем топ 10 похожих авторов, сохраним их имена в виде слоавря, так же для telegram бота.

In [27]:
authors_embed = {}

for author in tqdm(full_dataset['Authors'].unique()):
    books_filter = full_dataset['Authors'] == author
    mean_embed = embeddings[books_filter].mean(axis=0)

    authors_embed[author] = mean_embed


index_to_author = {i: key for i, key in enumerate(authors_embed.keys())}
author_to_index = {key: i for i, key in enumerate(authors_embed.keys())}

authors_matrix = np.array([embed for embed in authors_embed.values()])

100%|██████████| 8422/8422 [00:56<00:00, 148.81it/s]


In [33]:
authors_distances = []
authors_near_indexes = []

for i, emb in tqdm(enumerate(authors_matrix)):
    distances_tmp = np.dot(authors_matrix, emb)
    indexes = np.argsort(distances_tmp)[::-1][:10]

    authors_distances.append(distances_tmp[indexes])
    authors_near_indexes.append(indexes)

8422it [00:21, 391.60it/s]


In [57]:
authors_dict = {}

for i, idxs in tqdm(enumerate(authors_near_indexes)):
    author_name = index_to_author[i]
    similar_authors = [index_to_author[i] for i in idxs if index_to_author[i] != author_name]

    authors_dict[author_name] = similar_authors

8422it [00:00, 85034.48it/s]


In [58]:
with open('result/authors_embed_v1.pkl', 'wb') as file:
    pickle.dump(authors_embed, file)

with open('result/authors_dict_v1.pkl', 'wb') as file:
    pickle.dump(authors_dict, file)

Проверим полученные результаты.

In [63]:
authors_dict['Stephen King']

['Tim Underwood',
 'Al Sarrantonio',
 'Robin Furth',
 'Keith R.A. DeCandido',
 'Lydia Davis',
 'Alexander Masters',
 'Benjamin Zephaniah',
 'Steve Toltz',
 'Douglas E. Winter',
 'Graham McNamee']

In [64]:
authors_dict['J.K. Rowling']

['Newt Scamander',
 'Marc Shapiro',
 'Melissa Anelli',
 'Roger Highfield',
 'Suzy Kline',
 'Camron Wright',
 'Allan Zola Kronzek',
 'David Colbert',
 'M.V. Carey']

Находятся авторы. которые участвовали и помогали исходным авторам \
Авторы, у которых книги похожи по именам/персонажам и т.п. \
Авторы, которые на самом деле по стилистике чем-то похожи \
Авторы приквелов, книгах, которые о данных книгах

Примеры авторов для проверки: 

Stephen King \
J.K. Rowling \
Arthur Conan Doyle \
Agatha Christie \
J.R.R. Tolkien \
Mark Twain \
William Shakespeare

Примеры книг для проверки:

Calvin and Hobbes \
The Hobbit and The Lord of the Rings \
It \
The Green Mile \
Harry Potter and the Prisoner of Azkaban \
The Adventures of Tom Sawyer \
The Adventures of Sherlock Holmes \
Romeo And Juliet

#### Оценим качество получившихся эмбедингов для поиска похожих книг.
Для этого возьмём небольшой датасет с оценками пользователей. \
Из них оставим только те книги, которые пользователи оценили положительно. \
Будем считать, что если две книги встретились у одного пользователя, то они условно "близки" друг к другу. \
В качестве метрики ранжирования возьмём усреднённый Precision@10.

In [7]:
user_rating = pd.concat([pd.read_csv(str(x.resolve())) for x in Path("data/").glob("user_*.csv")])
user_rating = pd.merge(user_rating, full_dataset[['Id', 'Name']], on='Name', suffixes=('', '_book'))

In [8]:
with open('result/authors_dict_v1.pkl', 'rb') as file:
    authors_dict = pickle.load(file)

with open('result/books_dict_v1.pkl', 'rb') as file:
    books_dict = pickle.load(file)

In [9]:
# Берём только позитивно оцененные книги
user_rating_pos = user_rating[user_rating['Rating'].isin(['it was amazing', 'really liked it', 'liked it'])]

In [10]:
'''
Собираем книги, которые встречались вместе у пользователей.
'''

book_pairs = {}

for user_id in tqdm(user_rating_pos['ID'].unique()):
    user_books = user_rating_pos[user_rating_pos['ID'] == user_id]['Name'].values

    for i, name1 in enumerate(user_books):
        for j, name2 in enumerate(user_books, i+1):
            
            if name1 > name2:
                pair_names = (name1, name2)
            else:
                pair_names = (name2, name1)

            if book_pairs.get(pair_names) is None:
                book_pairs[pair_names] = 1
            else:
                book_pairs[pair_names] += 1

100%|██████████| 3734/3734 [02:08<00:00, 28.96it/s] 


In [11]:
for name in user_rating_pos['Name'].unique():
    del book_pairs[(name, name)]

Рассчитаем среднее значение Precision@10

In [12]:
# Создаем словарь с книгами, с совместной встречаемостью.

single_book_dict = {}
for name1, name2 in book_pairs.keys():

    if single_book_dict.get(name1) is None:
        single_book_dict[name1] = [name2]
    else:
        single_book_dict[name1].append(name2)

    if single_book_dict.get(name2) is None:
        single_book_dict[name2] = [name1]
    else:
        single_book_dict[name2].append(name1)

In [13]:
single_book_dict['Freakonomics: A Rogue Economist Explores the Hidden Side of Everything'][:10]

["The Restaurant at the End of the Universe (Hitchhiker's Guide to the Galaxy, #2)",
 'Siddhartha',
 'The Hunger Games (The Hunger Games, #1)',
 'The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury',
 'The Return of the Indian (The Indian in the Cupboard, #2)',
 'The Name of the Rose',
 'Dark Apprentice (Star Wars: The Jedi Academy Trilogy, #2)',
 'A Short History of Nearly Everything',
 'Angels & Demons (Robert Langdon, #1)',
 'The Return of the King (The Lord of the Rings, #3)']

In [14]:
'''
Посчитаем средний Precision@10
'''

precision_sum = 0
precision_cnt = 0

for name, sim_books in tqdm(books_dict.items()):
    if single_book_dict.get(name) is None:
        continue
    
    K = min(10, len(single_book_dict[name]))
    intersetction = len(set(single_book_dict[name]).intersection(set(sim_books[:K])))

    precision_sum += float(intersetction) / K
    precision_cnt += 1

100%|██████████| 36519/36519 [00:00<00:00, 47491.58it/s]


In [15]:
print(f'Среднее значение Precision@10 = {round(precision_sum / precision_cnt, 3)}')

Среднее значение Precision@10 = 0.128


#### Нечёткий поиск для опечаток в telegrem боте.
Возможна ситуация, когда пользователь телеграм бота неправильно ввёл название книги или имя автора. \
В этом случае воспользуемся библиотекой RapidFuzz https://github.com/maxbachmann/RapidFuzz для поиска похожих наименований. \
Данная библиотека позволяет очень быстро находить матчи. Что достаточно важно для пользовательского опыта \
Встроим эту систему в ТГ бот.

In [38]:
from rapidfuzz import process, fuzz

In [37]:
books_names = list(books_dict.keys())
authors_names = list(authors_dict.keys())

len(books_names), len(authors_names)

(36519, 8422)

In [39]:
%%timeit

query = 'Haroun and Sea stories'.lower()
res = process.extract(query, books_names, limit=5)

269 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
res

[('Haroun and the Sea of Stories', 95.0, 0),
 ("There and Back Again: An Actor's Tale", 85.5, 1),
 ('Rush Limbaugh Is a Big Fat Idiot and Other Observations (Americana)',
  85.5,
  26),
 ('Chinese Cinderella and the Secret Dragon Society: By the Author of Chinese Cinderella',
  85.5,
  76),
 ("The Girls' Guide to Hunting and Fishing", 85.5, 80)]